# RDS Data API - Row-level security using explicit transactions

To run this workbook, you must have an Amazon Aurora PostgreSQL cluster with the RDS Data API enabled. Update the cell below with the Cluster ARN, the Secrets Manager secret ARNs for the database credentials to use and the AWS Region to create the boto3 client in.

Note that the app user should not be the RDS master user as the row-level security policy will not apply to a super user. Instead create a dedicated application user to use with limited permissions.

In [ ]:
%pip install boto3

In [ ]:
import boto3

cluster_arn = '<cluster arn>'
admin_secret_arn = '<master user secret arn>'
app_user_secret_arn = '<app user secret arn>'
rdsData = boto3.client('rds-data', region_name='eu-central-1')
db_name = 'postgres'

The below cell creates a simple multi-tenant database schema, isolated using a row-level security policy:

In [ ]:
response = rdsData.execute_statement(resourceArn=cluster_arn,
                                      secretArn=admin_secret_arn,
                                      database=db_name,
                                      sql='CREATE TABLE tenant ( tenant_id integer PRIMARY KEY, tenant_name text, account_balance numeric );')
response = rdsData.execute_statement(resourceArn=cluster_arn,
                                      secretArn=admin_secret_arn,
                                      database=db_name,
                                      sql='''INSERT INTO tenant VALUES (1, 'Tenant1', 50000), (2, 'Tenant2', 60000), (3, 'Tenant3', 40000);''')
response = rdsData.execute_statement(resourceArn=cluster_arn,
                                      secretArn=admin_secret_arn,
                                      database=db_name,
                                      sql='''CREATE POLICY tenant_policy ON tenant USING (tenant_id = current_setting('tenant.id')::integer);''')
response = rdsData.execute_statement(resourceArn=cluster_arn,
                                      secretArn=admin_secret_arn,
                                      database=db_name,
                                      sql='ALTER TABLE tenant enable row level security;')

Finally, the below code can be used to create an explicit transaction in the application data access layer. This allows for set options to persist until the transaction is committed by reusing the same connection. The tenantId is pulled from the JWT as part of the tenant context. In this case it is hardcoded to "2", in a real-world application this would have logic to parse the tenantId from a claim in the token. 

In [ ]:
def get_tenant_id_from_context():
    return 2;

tr = rdsData.begin_transaction(
     resourceArn = cluster_arn,
     secretArn = app_user_secret_arn,
     database = db_name)

rdsData.execute_statement(resourceArn=cluster_arn,
                          secretArn=app_user_secret_arn,
                          database=db_name,
                          sql='set tenant.id = {0}'.format(get_tenant_id_from_context()),
                          transactionId = tr['transactionId'])

response = rdsData.execute_statement(resourceArn=cluster_arn,
                                      secretArn=app_user_secret_arn,
                                      database=db_name,
                                      sql='select tenant_name from tenant',
                                      transactionId = tr['transactionId'])

print(response['records'])

cr = rdsData.commit_transaction(
     resourceArn = cluster_arn,
     secretArn = app_user_secret_arn,
     transactionId = tr['transactionId'])